#  Deaths heatmap
---

Inspired by: https://twitter.com/VictimOfMaths/status/1258783876870176770?s=09

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-packages" data-toc-modified-id="Import-packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import packages</a></span></li><li><span><a href="#Import-data" data-toc-modified-id="Import-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import data</a></span></li><li><span><a href="#States-heatmap" data-toc-modified-id="States-heatmap-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>States heatmap</a></span></li><li><span><a href="#Cities-heatmap" data-toc-modified-id="Cities-heatmap-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Cities heatmap</a></span></li></ul></div>

## Import packages

In [3]:
import sys
sys.path.append("..")
sys.path.append("../src")

import yaml
config = yaml.load(open('../src/configs/config.yaml', 'r'))

/Users/fernandascovino/Projects/qualsetorreabrir/venvanalysis/lib/python3.7/site-packages/ipykernel_launcher.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [4]:
%reload_ext autoreload
%autoreload 2

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH, THEMES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999

import warnings

warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

# Setting cufflinks
import textwrap
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# Centering and fixing title
def iplottitle(title, width=40):
    return '<br>'.join(textwrap.wrap(title, width))

# Adding custom colorscales (to add one: themes/custom_colorscales.yaml)
import yaml
custom_colorscales = yaml.load(open(THEMES_PATH / 'custom_colorscales.yaml', 'r'))
cf.colors._custom_scales['qual'].update(custom_colorscales)
cf.colors.reset_scales()

# Setting cuffilinks template (use it with .iplot(theme='custom')
cf.themes.THEMES['custom'] = yaml.load(open(THEMES_PATH / 'cufflinks_template.yaml', 'r'))

## Import data

In [7]:
df = pd.read_csv('data/raw/br_cases_city_daily_notification_rate.csv', index_col=0)
df

,city,city_id,last_updated,epidemiological_week,estimated_population_2019,is_last,is_repeated,confirmed_cases,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,deaths,order_for_place,place_type,state,daily_cases,new_deaths,infectious_period_cases,state_notification_rate,notification_rate,active_cases
0,São Paulo,3550308,2020-02-25,9,12252023.0,False,False,1,0.00816,2020-02-25,0.0000,0,1,city,SP,1,0,1.0,NaN,NaN,NaN
1,São Paulo,3550308,2020-02-26,9,12252023.0,False,False,1,0.00816,2020-02-26,0.0000,0,2,city,SP,0,0,1.0,NaN,NaN,NaN
2,São Paulo,3550308,2020-02-27,9,12252023.0,False,False,1,0.00816,2020-02-27,0.0000,0,3,city,SP,0,0,1.0,NaN,NaN,NaN
3,São Paulo,3550308,2020-02-28,9,12252023.0,False,False,2,0.01632,2020-02-28,0.0000,0,4,city,SP,1,0,2.0,NaN,NaN,NaN
4,São Paulo,3550308,2020-02-29,9,12252023.0,False,False,2,0.01632,2020-02-29,0.0000,0,5,city,SP,0,0,2.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47708,Uruaçu,5221601,2020-05-04,19,40532.0,False,True,2,4.93437,2020-05-03,0.0000,0,23,city,GO,0,0,2.0,0.551335,0.551335,4.0
47709,Urutaí,5221809,2020-05-04,19,3072.0,False,True,0,0.00000,2020-05-03,0.0000,0,8,city,GO,0,0,0.0,0.551335,0.551335,0.0
47710,Valparaíso de Goiás,5221858,2020-05-04,19,168468.0,False,True,17,10.09094,2020-05-03,0.0588,1,42,city,GO,0,0,10.0,0.551335,0.288571,35.0
47711,Vianópolis,5222005,2020-05-04,19,13863.0,False,True,1,7.21345,2020-05-03,0.0000,0,28,city,GO,0,0,0.0,0.551335,0.551335,0.0


## States heatmap

In [13]:
def get_rolling_amount(grp):
    return grp.rolling(5, min_periods=1, on='last_updated')['deaths'].mean()

df_deaths = df[~df['deaths'].isnull()][['state', 'last_updated', 'deaths']]\
                                                .groupby(['state', 'last_updated'])['deaths']\
                                                .sum()\
                                                .reset_index()

df_deaths['rolling_deaths_new'] = df_deaths.groupby('state', 
                                                    as_index=False, 
                                                    group_keys=False)\
                                           .apply(get_rolling_amount)

In [14]:
df_deaths

,state,last_updated,deaths,rolling_deaths_new
0,AC,2020-03-17,0,0.0
1,AC,2020-03-18,0,0.0
2,AC,2020-03-19,0,0.0
3,AC,2020-03-20,0,0.0
4,AC,2020-03-21,0,0.0
...,...,...,...,...
1410,TO,2020-04-30,3,2.4
1411,TO,2020-05-01,4,2.8
1412,TO,2020-05-02,4,3.2
1413,TO,2020-05-03,4,3.6


In [18]:
def _df_to_plotly(df):
    return {'z': df.values.tolist(),
            'x': df.columns.tolist(),
            'y': df.index.tolist()}
    
def plot_deaths_heatmap(t, place_type, title):
    
    df_heatmap = t.reset_index()\
                          .pivot(index=place_type, 
                                  columns='last_updated', 
                                  values='rolling_deaths_new')\
                           .fillna(0)\
                           .apply(lambda x: x/x.max(), axis=1)\
                           .dropna(how='all')

    # remove days with all states zero
    df_heatmap = df_heatmap.loc[:, (df_heatmap != 0).any(axis=0)]
    print(len(df_heatmap))

    fig = go.Figure(data=go.Heatmap(_df_to_plotly(df_heatmap), 
                                    colorscale='temps'), 
                    layout={'width': 600,
                            'height': 800,
                            'title': title})
    fig.show()

In [19]:
plot_deaths_heatmap(df_deaths, 'state',
                    title='Distribuição de novas mortes nas UFs (mavg = 5 days)')

27


## Cities heatmap

In [ ]:
df_heatmap = state_df_deaths.reset_index()\
                      .pivot(index='state', 
                              columns='date', 
                              values='rolling_deaths_new')\
                       .fillna(0)\
                       .apply(lambda x: x/x.max(), axis=1)

# remove days with all states zero
df_heatmap = df_heatmap.loc[:, (df_heatmap != 0).any(axis=0)]

def df_to_plotly(df):
    return {'z': df.values.tolist(),
            'x': df.columns.tolist(),
            'y': df.index.tolist()}

fig = go.Figure(data=go.Heatmap(df_to_plotly(df_heatmap), colorscale='temps'), 
                layout={'width': 600,
                        'height': 600,
                        'title': 'Distribuição de novas mortes nas UFs (mavg = 5 days)'})
fig.show()

In [20]:
state = 'RS'

In [21]:
df_deaths = df[(~df['deaths'].isnull()) & (df['state'] == state)][['city', 
                                                                 'last_updated', 
                                                                 'deaths']]

df_deaths['rolling_deaths_new'] = df_deaths.groupby('city', 
                                                as_index=False, 
                                                group_keys=False)\
                                          .apply(get_rolling_amount)

In [23]:
plot_deaths_heatmap(df_deaths, 'city', 'Distribuição de novas mortes em {}'.format(state))

30
